# Crawlinng Berita Ridwan Kamil

In [1]:
import pandas as pd
import requests as re
from bs4 import BeautifulSoup as bs
from urllib.request import Request, urlopen
import csv

In [2]:
header = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36"}
url = 'https://www.detik.com/search/searchall?query=pilpres+2024&siteid=2'

judul = []
konten = []

for page in range(1,11):
  req = re.get(url+str(page), headers=header)
  soup = bs(req.text, 'html.parser')
  items = soup.findAll('article')
  for it in items:
    try : link = it.find('a')['href']
    except : link =''
    name = it.find('h2', 'title').text
    judul.append(name)

    if link != '':
      req2 = re.get(str(link), headers=header)
      soup2 = bs(req2.text, 'html.parser')
      items2 = soup2.findAll("div", "column-8")
      for it2 in items2:
        isi = it2.find('p').text

        if isi != '':
          konten.append(isi)


JudulBerita = pd.DataFrame(judul, columns=["Judul Berita"])
IsiBerita = pd.DataFrame(konten, columns=["Isi Berita"])

data = pd.concat([JudulBerita, IsiBerita], axis=1)
data = data.loc[0:50]
data

,Judul Berita,Isi Berita
0,Pengamat: Politik Erat Kaitannya dengan Olahraga,\r\nADVERTISEMENT\r\n
1,Podcast: Menaksir Peluang Gol Janji Surga Capr...,Tiga pasangan terpilih akhirnya resmi mendafta...
2,Nasib IKN di Tangan Ganjar-Mahfud & Prabowo-Gi...,Sebanyak tiga pasangan capres-cawapres telah m...
3,Apa Itu Dana Abadi yang Dibicarakan Gibran?,Cawapres Gibran Rakabuming Raka telah mencanan...
4,Ini Program Dana Abadi & KIS Lansia Prabowo-Gi...,Pasangan capres-cawapres dari Koalisi Indonesi...
5,Cek di Sini! Dana Abadi Pesantren yang Dijanji...,Cawapres Gibran Rakabuming Raka sempat membebe...
6,Gibran Beberkan KIS Lansia-Dana Abadi Pesantre...,"Gibran Rakabuming Raka, Cawapres Prabowo Subia..."
7,"Gabung TPN Ganjar-Mahfud, Eko Sulistyo Mundur ...",Komisaris PT PLN (Persero) Eko Sulistyo menyeb...
8,Kemenkeu soal KIS Lansia Besutan Gibran: Harus...,Kementerian Keuangan (Kemenkeu) menanggapi sal...
9,Erick Thohir Bertemu Sandiaga Saat Probowo-Gib...,Menteri BUMN Erick Thohir bertemu dengan Mente...


In [3]:
data.to_csv('berita.csv', index=False)

In [4]:
df = pd.read_csv('berita.csv')
df

,Judul Berita,Isi Berita
0,Pengamat: Politik Erat Kaitannya dengan Olahraga,\r\nADVERTISEMENT\r\n
1,Podcast: Menaksir Peluang Gol Janji Surga Capr...,Tiga pasangan terpilih akhirnya resmi mendafta...
2,Nasib IKN di Tangan Ganjar-Mahfud & Prabowo-Gi...,Sebanyak tiga pasangan capres-cawapres telah m...
3,Apa Itu Dana Abadi yang Dibicarakan Gibran?,Cawapres Gibran Rakabuming Raka telah mencanan...
4,Ini Program Dana Abadi & KIS Lansia Prabowo-Gi...,Pasangan capres-cawapres dari Koalisi Indonesi...
5,Cek di Sini! Dana Abadi Pesantren yang Dijanji...,Cawapres Gibran Rakabuming Raka sempat membebe...
6,Gibran Beberkan KIS Lansia-Dana Abadi Pesantre...,"Gibran Rakabuming Raka, Cawapres Prabowo Subia..."
7,"Gabung TPN Ganjar-Mahfud, Eko Sulistyo Mundur ...",Komisaris PT PLN (Persero) Eko Sulistyo menyeb...
8,Kemenkeu soal KIS Lansia Besutan Gibran: Harus...,Kementerian Keuangan (Kemenkeu) menanggapi sal...
9,Erick Thohir Bertemu Sandiaga Saat Probowo-Gib...,Menteri BUMN Erick Thohir bertemu dengan Mente...


In [5]:
!pip install Sastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.1 MB/s eta 0:00:00


In [6]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

import warnings
import pandas as pd
import numpy as np
import nltk
import re
import csv

nltk.download('stopwords')
nltk.download('punkt')
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## 1. Cleaning Data

In [7]:
df.isnull().sum()

Judul Berita    0
Isi Berita      0
dtype: int64

In [8]:
df = df.dropna()
df.isnull().sum()

Judul Berita    0
Isi Berita      0
dtype: int64

In [9]:
def cleaning(text):
  text = re.sub(r'[^a-zA-Z\s]', '', text).strip()
  return text

df['Cleaning'] = df['Isi Berita'].apply(cleaning)
df['Cleaning']

0                                        ADVERTISEMENT
1    Tiga pasangan terpilih akhirnya resmi mendafta...
2    Sebanyak tiga pasangan caprescawapres telah me...
3    Cawapres Gibran Rakabuming Raka telah mencanan...
4    Pasangan caprescawapres dari Koalisi Indonesia...
5    Cawapres Gibran Rakabuming Raka sempat membebe...
6    Gibran Rakabuming Raka Cawapres Prabowo Subian...
7    Komisaris PT PLN Persero Eko Sulistyo menyebut...
8    Kementerian Keuangan Kemenkeu menanggapi salah...
9    Menteri BUMN Erick Thohir bertemu dengan Mente...
Name: Cleaning, dtype: object

In [10]:
def cek_specialCharacter(dokumen):
  karakter = ['!', '@', '#', '$', '%', '^', '&', '*', '(', ')', '-', '_', '+', '=', '{', '}', '[', ']', '|', '\\', ':', ';', '"', "'", '<', '>', ',', '.', '?', '/', '`', '~']
  for i in dokumen:
    if i in karakter :
      print(dokumen)
df['Cleaning'].apply(cek_specialCharacter)

0    None
1    None
2    None
3    None
4    None
5    None
6    None
7    None
8    None
9    None
Name: Cleaning, dtype: object

## 2. Tokenizing

In [11]:
def tokenizer(text):
  text = text.lower()
  return word_tokenize(text)

df['Tokenizing'] = df['Cleaning'].apply(tokenizer)
df['Tokenizing']

0                                      [advertisement]
1    [tiga, pasangan, terpilih, akhirnya, resmi, me...
2    [sebanyak, tiga, pasangan, caprescawapres, tel...
3    [cawapres, gibran, rakabuming, raka, telah, me...
4    [pasangan, caprescawapres, dari, koalisi, indo...
5    [cawapres, gibran, rakabuming, raka, sempat, m...
6    [gibran, rakabuming, raka, cawapres, prabowo, ...
7    [komisaris, pt, pln, persero, eko, sulistyo, m...
8    [kementerian, keuangan, kemenkeu, menanggapi, ...
9    [menteri, bumn, erick, thohir, bertemu, dengan...
Name: Tokenizing, dtype: object

In [12]:
def count_word(dokumens):
  return len(dokumens)

df['Count Word'] = df['Tokenizing'].apply(count_word)
df

,Judul Berita,Isi Berita,Cleaning,Tokenizing,Count Word
0,Pengamat: Politik Erat Kaitannya dengan Olahraga,\r\nADVERTISEMENT\r\n,ADVERTISEMENT,[advertisement],1
1,Podcast: Menaksir Peluang Gol Janji Surga Capr...,Tiga pasangan terpilih akhirnya resmi mendafta...,Tiga pasangan terpilih akhirnya resmi mendafta...,"[tiga, pasangan, terpilih, akhirnya, resmi, me...",25
2,Nasib IKN di Tangan Ganjar-Mahfud & Prabowo-Gi...,Sebanyak tiga pasangan capres-cawapres telah m...,Sebanyak tiga pasangan caprescawapres telah me...,"[sebanyak, tiga, pasangan, caprescawapres, tel...",38
3,Apa Itu Dana Abadi yang Dibicarakan Gibran?,Cawapres Gibran Rakabuming Raka telah mencanan...,Cawapres Gibran Rakabuming Raka telah mencanan...,"[cawapres, gibran, rakabuming, raka, telah, me...",32
4,Ini Program Dana Abadi & KIS Lansia Prabowo-Gi...,Pasangan capres-cawapres dari Koalisi Indonesi...,Pasangan caprescawapres dari Koalisi Indonesia...,"[pasangan, caprescawapres, dari, koalisi, indo...",42
5,Cek di Sini! Dana Abadi Pesantren yang Dijanji...,Cawapres Gibran Rakabuming Raka sempat membebe...,Cawapres Gibran Rakabuming Raka sempat membebe...,"[cawapres, gibran, rakabuming, raka, sempat, m...",27
6,Gibran Beberkan KIS Lansia-Dana Abadi Pesantre...,"Gibran Rakabuming Raka, Cawapres Prabowo Subia...",Gibran Rakabuming Raka Cawapres Prabowo Subian...,"[gibran, rakabuming, raka, cawapres, prabowo, ...",28
7,"Gabung TPN Ganjar-Mahfud, Eko Sulistyo Mundur ...",Komisaris PT PLN (Persero) Eko Sulistyo menyeb...,Komisaris PT PLN Persero Eko Sulistyo menyebut...,"[komisaris, pt, pln, persero, eko, sulistyo, m...",30
8,Kemenkeu soal KIS Lansia Besutan Gibran: Harus...,Kementerian Keuangan (Kemenkeu) menanggapi sal...,Kementerian Keuangan Kemenkeu menanggapi salah...,"[kementerian, keuangan, kemenkeu, menanggapi, ...",36
9,Erick Thohir Bertemu Sandiaga Saat Probowo-Gib...,Menteri BUMN Erick Thohir bertemu dengan Mente...,Menteri BUMN Erick Thohir bertemu dengan Mente...,"[menteri, bumn, erick, thohir, bertemu, dengan...",35


## 3. Stopword

In [13]:
corpus = stopwords.words('indonesian')

def stopwordText(words):
 return [word for word in words if word not in corpus]

df['Stopword Removal'] = df['Tokenizing'].apply(stopwordText)

# Gabungkan kembali token menjadi kalimat utuh
df['Full Text'] = df['Stopword Removal'].apply(lambda x: ' '.join(x))
df['Full Text']

0                                        advertisement
1    pasangan terpilih resmi mendaftarkan calon pre...
2    pasangan caprescawapres menyusun visi misi pro...
3    cawapres gibran rakabuming raka mencanangkan d...
4    pasangan caprescawapres koalisi indonesia maju...
5    cawapres gibran rakabuming raka membeberkan pr...
6    gibran rakabuming raka cawapres prabowo subian...
7    komisaris pt pln persero eko sulistyo menyebut...
8    kementerian keuangan kemenkeu menanggapi salah...
9    menteri bumn erick thohir bertemu menteri pari...
Name: Full Text, dtype: object

## 4. One Hot Encoding

In [14]:
def pandasOneHotEncoder(dokumens):
  encoder  = pd.get_dummies(dokumens.apply(pd.Series).stack()).sum(level=0)
  df = pd.concat([dokumens, encoder], axis=1)

  return df

oneHotEncoder = pandasOneHotEncoder(df['Stopword Removal'])
oneHotEncoder

,Stopword Removal,abadi,advertisement,bertarung,bertemu,bumn,calon,capres,caprescawapres,cawapres,...,thohir,tim,timur,tpn,unggulan,unggulannya,uno,visi,visimisi,wakil
0,[advertisement],0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"[pasangan, terpilih, resmi, mendaftarkan, calo...",0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,1
2,"[pasangan, caprescawapres, menyusun, visi, mis...",0,0,1,0,0,0,0,1,0,...,0,0,1,0,0,0,0,1,0,0
3,"[cawapres, gibran, rakabuming, raka, mencanang...",2,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
4,"[pasangan, caprescawapres, koalisi, indonesia,...",1,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
5,"[cawapres, gibran, rakabuming, raka, membeberk...",1,0,0,0,0,0,0,0,1,...,0,0,0,0,1,1,0,0,0,0
6,"[gibran, rakabuming, raka, cawapres, prabowo, ...",1,0,0,0,0,0,0,0,1,...,0,0,0,0,2,0,0,0,0,0
7,"[komisaris, pt, pln, persero, eko, sulistyo, m...",0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,1
8,"[kementerian, keuangan, kemenkeu, menanggapi, ...",1,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
9,"[menteri, bumn, erick, thohir, bertemu, menter...",0,0,0,1,1,0,1,0,1,...,1,0,0,0,0,0,1,0,0,0


## 5. TF IDF

In [15]:
def tfidf(dokumen):
  vectorizer = TfidfVectorizer()
  x = vectorizer.fit_transform(dokumen).toarray()
  terms = vectorizer.get_feature_names_out()

  final_tfidf = pd.DataFrame(x, columns=terms)
  final_tfidf.insert(0, 'Dokumen', dokumen)

  return (vectorizer, final_tfidf)

tfidf_vectorizer, final_tfidf = tfidf(df['Full Text'])
final_tfidf

,Dokumen,abadi,advertisement,bertarung,bertemu,bumn,calon,capres,caprescawapres,cawapres,...,thohir,tim,timur,tpn,unggulan,unggulannya,uno,visi,visimisi,wakil
0,advertisement,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,pasangan terpilih resmi mendaftarkan calon pre...,0.000000,0.0,0.000000,0.000000,0.000000,0.275682,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.275682,0.234355
2,pasangan caprescawapres menyusun visi misi pro...,0.000000,0.0,0.184661,0.000000,0.000000,0.000000,0.000000,0.211070,0.000000,...,0.000000,0.000000,0.248291,0.000000,0.000000,0.000000,0.000000,0.211070,0.000000,0.000000
3,cawapres gibran rakabuming raka mencanangkan d...,0.337330,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.187811,...,0.000000,0.000000,0.000000,0.000000,0.187811,0.000000,0.000000,0.000000,0.000000,0.000000
4,pasangan caprescawapres koalisi indonesia maju...,0.137216,0.0,0.171856,0.000000,0.000000,0.000000,0.000000,0.196433,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.196433,0.000000,0.000000
5,cawapres gibran rakabuming raka membeberkan pr...,0.192010,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.213806,...,0.000000,0.000000,0.000000,0.000000,0.213806,0.323346,0.000000,0.000000,0.000000,0.000000
6,gibran rakabuming raka cawapres prabowo subian...,0.172418,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.191990,...,0.000000,0.000000,0.000000,0.000000,0.383980,0.000000,0.000000,0.000000,0.000000,0.000000
7,komisaris pt pln persero eko sulistyo menyebut...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.214557,0.000000,0.214557,0.000000,0.000000,0.000000,0.000000,0.000000,0.182393
8,kementerian keuangan kemenkeu menanggapi salah...,0.161883,0.0,0.202750,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.180259,0.000000,0.000000,0.000000,0.000000,0.000000
9,menteri bumn erick thohir bertemu menteri pari...,0.000000,0.0,0.000000,0.223692,0.223692,0.000000,0.223692,0.000000,0.147912,...,0.223692,0.000000,0.000000,0.000000,0.000000,0.000000,0.223692,0.000000,0.000000,0.000000


In [16]:
def term_freq(dokumens):
  # Buat objek CountVectorizer
  vectorizer = CountVectorizer()
  tf_matrix = vectorizer.fit_transform(dokumens).toarray()
  terms = vectorizer.get_feature_names_out()

  final_tf = pd.DataFrame(tf_matrix, columns=terms)
  final_tf.insert(0, 'Dokumen', dokumens)

  return (vectorizer, final_tf, tf_matrix, terms)

tf_vectorizer, final_tf, tf_matrix, tf_terms = term_freq(df['Full Text'])
final_tf

,Dokumen,abadi,advertisement,bertarung,bertemu,bumn,calon,capres,caprescawapres,cawapres,...,thohir,tim,timur,tpn,unggulan,unggulannya,uno,visi,visimisi,wakil
0,advertisement,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,pasangan terpilih resmi mendaftarkan calon pre...,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,1
2,pasangan caprescawapres menyusun visi misi pro...,0,0,1,0,0,0,0,1,0,...,0,0,1,0,0,0,0,1,0,0
3,cawapres gibran rakabuming raka mencanangkan d...,2,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
4,pasangan caprescawapres koalisi indonesia maju...,1,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
5,cawapres gibran rakabuming raka membeberkan pr...,1,0,0,0,0,0,0,0,1,...,0,0,0,0,1,1,0,0,0,0
6,gibran rakabuming raka cawapres prabowo subian...,1,0,0,0,0,0,0,0,1,...,0,0,0,0,2,0,0,0,0,0
7,komisaris pt pln persero eko sulistyo menyebut...,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,1
8,kementerian keuangan kemenkeu menanggapi salah...,1,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
9,menteri bumn erick thohir bertemu menteri pari...,0,0,0,1,1,0,1,0,1,...,1,0,0,0,0,0,1,0,0,0


In [17]:
new = final_tf.drop(columns=["Dokumen"])
new

,abadi,advertisement,bertarung,bertemu,bumn,calon,capres,caprescawapres,cawapres,dana,...,thohir,tim,timur,tpn,unggulan,unggulannya,uno,visi,visimisi,wakil
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
2,0,0,1,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,1,0,0
3,2,0,0,0,0,0,0,0,1,2,...,0,0,0,0,1,0,0,0,0,0
4,1,0,1,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,1,0,0
5,1,0,0,0,0,0,0,0,1,1,...,0,0,0,0,1,1,0,0,0,0
6,1,0,0,0,0,0,0,0,1,1,...,0,0,0,0,2,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,1
8,1,0,1,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
9,0,0,0,1,1,0,1,0,1,0,...,1,0,0,0,0,0,1,0,0,0


In [18]:
from sklearn.metrics.pairwise import cosine_similarity

tested=pd.DataFrame(cosine_similarity(new, dense_output=True))

tested.head()

,0,1,2,3,4,5,6,7,8,9
0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.0,1.000000,0.187317,0.042601,0.155113,0.146735,0.000000,0.048912,0.176604,0.046829
2,0.0,0.187317,1.000000,0.151620,0.483046,0.348155,0.200160,0.000000,0.392837,0.041667
3,0.0,0.042601,0.151620,1.000000,0.470824,0.791808,0.655521,0.000000,0.500319,0.189525
4,0.0,0.155113,0.483046,0.470824,1.000000,0.540562,0.662994,0.000000,0.683130,0.138013
